# Imports, Functions, and Variables

In [1]:
import pandas as pd
import numpy as np

# data_file = r"Example Activity Report Generated by OpenLM 1day.csv"
data_file = r"Example Activity Report Generated by OpenLM NOVtoFEB.csv"
# data_file = r"Example Activity Report Generated by OpenLM SEPTtoFEB.csv"

fields_to_drop = ["Version", "License Type", "Borrowed", "Server", "Vendor", "Additional Key", "Host Ids", "IP",
                  "Project", "Group", "Usage Time w/in filter period", "Consumed Tokens", "Idle Time (hours)",
                  "Token Usage Time", "Token Usage Time w/in filter period", "Session ID",
                  "Source"]
substitution_file = r"WorkstationToAgency_SubstitutionsList.csv"

# Process Data
### Create Master Dataframe from OpenLM Report (.csv) of Interest


In [2]:
master_df = pd.read_csv(filepath_or_buffer=data_file)
# master_df.info()

### Slim Size of Dataframe by Dropping Unneeded Fields

In [3]:
master_df.drop(columns=fields_to_drop, inplace=True)

### Field Calculations
NOTE: When i try to use f strings or .format to form the concatenated values I encounter weird results.

#### Get Workstation Substitution Values as a Python Dictionary

In [4]:
substitutions_df = pd.read_csv(filepath_or_buffer=substitution_file)
substitutions_df.set_index("VALUE", inplace=True)
substitutions_dict = substitutions_df.to_dict(orient="dict")["REPLACEMENT"]

#### Add New Fields and Calculate

In [6]:
# Calculate Product_Workstation field
master_df["Product_Workstation"] = master_df["Product"] + "_" + master_df["Workstation"]

# Calculate Product_Username field
master_df["Product_Username"] = master_df["Product"] + "_" + master_df["User Name"]

# Convert Start and End Time fields
master_df["Start Time"] = pd.to_datetime(arg=master_df["Start Time"], dayfirst=True)
master_df["End Time"] = pd.to_datetime(arg=master_df["End Time"], dayfirst=True)

# Calculate Date field from datetime object. For Peak Usage Report
master_df["Date"] = master_df["End Time"].apply(lambda x: x.strftime("%Y%m%d"))

In [7]:
# Calculate Agency field
def workstation_to_agency(workstation_value):
    result = "Research"
    for key, value in substitutions_dict.items():
        if key in workstation_value:
            result = value
            break
        else:
            continue
    return result

master_df["Agency"] = master_df["Workstation"].apply(func=workstation_to_agency)
# master_df.info()

## Quick Evaluations
#### Unique 'Agency' values and counts

In [ ]:
# master_df["Agency"].value_counts()

#### See Full Records for where 'Agency' equals 'Research'

In [ ]:
# master_df[(master_df["Agency"] == "Research")][["Product_Workstation", "Agency"]]

#### See Workstation Substitutions Values

In [ ]:
# substitutions_dict

# Report Generation
## Product_Workstation Report

NOTE: The .agg() function is applied to entire dataframe but only numeric fields are evaluated. 
The 'Total usage time (hours)' column is the only numeric column. The output is what we are seeking
but if another numeric field was added this would need to be revised.

In [8]:
agency_product_workstation_gbdf = master_df.groupby(by=["Agency", "Product_Workstation"], axis=0)
agency_product_workstation_usage_df = agency_product_workstation_gbdf.agg(np.sum)
agency_product_workstation_count = agency_product_workstation_gbdf["Product_Workstation"].agg(np.count_nonzero)
agency_product_workstation_count.name = "Frequency"
product_workstation_report = agency_product_workstation_usage_df.join(other=agency_product_workstation_count, how="left")

## Product_Username Report

In [9]:
agency_product_username_gbdf = master_df.groupby(by=["Agency", "Product_Username"], axis=0)
agency_product_username_usage_df = agency_product_username_gbdf.agg(np.sum)
agency_product_username_count = agency_product_username_gbdf["Product_Username"].agg(np.count_nonzero)
agency_product_username_count.name = "Frequency"
product_username_report = agency_product_username_usage_df.join(other=agency_product_username_count, how="left")

## Evaluation of Product Use by Agency 

TODO: Need to add the ProductLabel field to these reports.

### Workstation and Username - Per Product Per Agency Report

In [16]:
agency_product_gbdf = master_df.groupby(by=["Agency", "Product"], axis=0)
agency_product_usage_df = agency_product_gbdf.agg(np.sum)
agency_product_count_series = agency_product_gbdf["Product"].agg(np.count_nonzero)
agency_product_count_series.name = "Frequency"

agency_username_count_series = agency_product_gbdf["User Name"].nunique() # for next one use workstation id
agency_username_count_series.name = "Unique Usernames"

# Code preserved in case wanted to produce two separate reports
# agency_product_count_report = agency_product_usage_df.join(other=agency_product_count_series, 
#                                                            how="left").join(other=agency_username_count_series, 
#                                                                             how="left")

agency_workstation_count_series = agency_product_gbdf["Workstation"].nunique() # for next one use workstation id
agency_workstation_count_series.name = "Unique Workstations"
agency_product_count_report = agency_product_usage_df.join(other=agency_product_count_series, 
                                                           how="left").join(other=agency_workstation_count_series, 
                                                                            how="left").join(other=agency_username_count_series, 
                                                                            how="left")
agency_product_count_report.head()

Total usage time (hours)  Frequency  \
Agency Product                                                
CHART  ArcInfo                             11.21       2402   
       Network Analyst                      0.42          4   
COMP   ArcInfo                            124.20       1171   
DBED   3DAnalyst                            2.00          1   
       ArcEditor                            4.37          6   

                        Unique Workstations  Unique Usernames  
Agency Product                                                 
CHART  ArcInfo                            2                 4  
       Network Analyst                    1                 1  
COMP   ArcInfo                            1                 5  
DBED   3DAnalyst                          1                 1  
       ArcEditor                          1                 1

## Peak Usage Report

In [ ]:
# master_df.info()

In [ ]:
peak_usage_gbdf = master_df.groupby(by=["Agency", "Date", "Product"], axis=0)
peak_usage_df = peak_usage_gbdf.agg(np.sum)
# peak_usage_df.head()

In [ ]:
peak_usage_report = peak_usage_df.groupby(by=["Agency", "Product"]).agg(np.max)

#### Inspect for Specific Agency Product Usage
NOTE: Use the below code to verify the findings above for each agencies max usage across all dates a product was used by that agency

In [ ]:
# agency_abbreviation_you_want_to_check = "SHA" # EDIT MY VALUE
# product_name_you_want_to_check = "ArcInfo" # EDIT MY VALUE
# try:
#     agency_isolation_df = peak_usage_df.xs(key=agency_abbreviation_you_want_to_check, axis=0, level=0)
#     product_isolation_series = agency_isolation_df.xs(key=product_name_you_want_to_check, axis=0, level=1).agg(np.max)
# except KeyError as ke:
#     print("Key Error occurred. {key} not found. Try again.".format(key=ke))
# product_isolation_series

## Output Reports to Excel File

In [ ]:
output_excel_file_name = "TEST_OUTPUT.xlsx"

In [ ]:
with pd.ExcelWriter(path=output_excel_file_name, datetime_format="YYYY-MM-DD") as writer:
    product_workstation_report.to_excel(writer, sheet_name="Product Workstation")
    product_username_report.to_excel(writer, sheet_name="Product Username")
    agency_product_count_report.to_excel(writer, sheet_name="Agency Product Count")
    agency_username_count_report.to_excel(writer, sheet_name="Agency Username")
    peak_usage_report.to_excel(writer, sheet_name="Peak Usage")

# General Statistics on Usage

In [ ]:
# master_df["Total usage time (hours)"].groupby(by=master_df["Agency"], axis=0).describe()

In [ ]:
# master_df["Total usage time (hours)"].groupby(by=master_df["Product"], axis=0).describe()